In [1]:
# ProcessIdentification class
# Necessary libraries
import numpy as np
import os
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")

class FirstOrderProcessIdentification:
    def __init__(self, n_hidden_layers=2, n_neurons=100, optimizer_class=tf.train.MomentumOptimizer,
                 momentum = 0.9, learning_rate=0.01, n_inputs = 11, n_outputs = 1, batch_size=50,
                 activation=tf.nn.relu, learning_object=None, loaded_data=None, random_state=None):
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.optimizer_class = optimizer_class
        self.momentum = momentum
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.activation = activation
        self.learning_object = learning_object
        self.loaded_data = loaded_data
        self.random_state = random_state
        self._session = None
        
    def data_preparation(self):
        train_set, test_set = train_test_split(self.loaded_data, test_size=0.2, random_state=42)
        X_train, y_train = train_set.iloc[:,:11].values, train_set.iloc[:,11:].values
        X_test, y_test = test_set.iloc[:,:11].values, test_set.iloc[:,11:].values
        X_scale_minmax = X_train
        scaler = MinMaxScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)   
        return X_train, y_train, X_test, y_test, X_scale_minmax
    
    def operating_data_preparation(self, X_scale_minmax):
        X_op, y_op = self.loaded_data.iloc[:,:11].values, self.loaded_data.iloc[:,11:].values
        process_gain = X_op[0,:]        
        scaler = MinMaxScaler()
        scaler.fit(X_scale_minmax)
        X_op = scaler.transform(X_op)
        return X_op, y_op, process_gain

    def _dnn(self, inputs):
        for layer in range(self.n_hidden_layers):
            inputs = tf.layers.dense(inputs, self.n_neurons, name="hidden%d" % (layer + 1))
            inputs = self.activation(inputs, name="hidden%d_out" % (layer + 1))
        return inputs
    
    def _build_graph(self, n_inputs, n_outputs):
        if self.random_state is not None:
            tf.set_random_seed(self.random_state)
            np.random.seed(self.random_state)
            
        X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
        y = tf.placeholder(tf.float32, shape=(None), name="y")
        
        dnn_outputs = self._dnn(X)
        
        y_predict = tf.layers.dense(dnn_outputs, n_outputs, name="outputs")
        
        RMSE = tf.sqrt(tf.losses.mean_squared_error(y, y_predict), name="loss")
        
        optimizer = self.optimizer_class(learning_rate=self.learning_rate, momentum=self.momentum, use_nesterov=True)
        training_op = optimizer.minimize(RMSE)
        
        accuracy = tf.reduce_mean(1-(abs(y_predict - y)/y), name="accuracy")
        
        init = tf.global_variables_initializer()
        saver = tf.train.Saver()
        
        self._X, self._y = X, y
        self._y_predict, self._RMSE = y_predict, RMSE
        self._training_op, self._accuracy = training_op, accuracy
        self._init, self._saver = init, saver

    def close_session(self):
        if self._session:
            self._session.close()

    def fit(self, X, y, n_epochs=20):
        self.close_session()
        
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(self.n_inputs, self.n_outputs)        
        
        self._session = tf.Session(graph=self._graph)
        
        root_logdir = "tf_logs_1st_%s_RMSE" % (self.learning_object)
        logdir = "{}/run-{}/".format(root_logdir, now)
        RMSE_summary = tf.summary.scalar('RMSE', self._RMSE)
        file_writer = tf.summary.FileWriter(logdir, self._session.graph)
        
        with self._session.as_default() as sess:
            self._init.run()
            for epoch in range(n_epochs):
                rnd_idx = np.random.permutation(len(X))
                for rnd_indices in np.array_split(rnd_idx, len(X) // self.batch_size):
                    X_batch, y_batch = X[rnd_indices], y[rnd_indices]
                    feed_dict = {self._X: X_batch, self._y: y_batch}
                    sess.run(self._training_op, feed_dict=feed_dict)
                acc_batch = self._accuracy.eval(feed_dict=feed_dict)
                RMSE_batch = self._RMSE.eval(feed_dict=feed_dict)
                summary_str = RMSE_summary.eval(feed_dict=feed_dict)
                file_writer.add_summary(summary_str, epoch)
                print("{}\tbatch data accuracy: {:.4f}\tbatch data RMSE: {:.6f}".format(epoch, acc_batch, RMSE_batch))
            
            save_path = self._saver.save(sess, "./1st_order_process_%s_learning.ckpt" % self.learning_object)
            
    def predict(self, X, y):
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(self.n_inputs, self.n_outputs)
            
        self._session = tf.Session(graph=self._graph)
        with self._session.as_default() as sess:
            self._saver.restore(sess, "./1st_order_process_%s_learning.ckpt" % self.learning_object)
            accuracy_val = self._accuracy.eval(feed_dict={self._X: X, self._y: y})
            X_new_scaled = X[:10]
            y_pred = self._y_predict.eval(feed_dict={self._X: X_new_scaled})
            
            print("\n")
            print("%s test data accuracy:" % self.learning_object, accuracy_val)
            print("\n")
        
        return y_pred
    
    def operating_predict(self, X, y):
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(self.n_inputs, self.n_outputs)
            
        self._session = tf.Session(graph=self._graph)
        with self._session.as_default() as sess:
            self._saver.restore(sess, "./1st_order_process_%s_learning.ckpt" % self.learning_object)
            X_new_scaled = X[:1]
            y_pred = self._y_predict.eval(feed_dict={self._X: X_new_scaled})

        return y_pred

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#PID Tuning class    
class PIDTuning:
    def __init__(self, tau=None, theta=None, PID_type=None):
        self.tau = tau
        self.theta = theta
        self.PID_type = PID_type         

    def IMC_tuning_PIPC(self, process_gain=None):
        lamda = 0.25 * self.theta
        if self.PID_type == "PI":
            controller_gain = ((2*self.tau + self.theta)/(2*lamda))/process_gain
            integral_time = self.tau + self.theta/2
            print("PI Controller by IMC Tuning_PIPC: Kc = %f, tau_i = %f" % (controller_gain, integral_time))
        elif self.PID_type == "PID":
            controller_gain = ((2*self.tau + self.theta)/(2*(lamda + self.theta)))/process_gain
            integral_time = self.tau + self.theta/2
            derivative_time = self.tau*self.theta/(2*self.tau + self.theta)
            print("PID Controller by IMC Tuning_PIPC: Kc = %f, tau_i = %f, tau_d = %f" %
                  (controller_gain, integral_time, derivative_time))
    
    def IMC_tuning_PDCT(self, process_gain=None, Condition=None):
        if Condition == "aggressive control":
            tau_c = 0.5/self.theta
        elif Condition == "default value":
            tau_c = self.theta
        elif Condition == "smooth control":
            tau_c = 1.5*self.theta
        elif Condition == "robust control":
            tau_c = 3*self.theta
        if self.PID_type == "PI":
            controller_gain = self.tau/((tau_c+self.theta)*process_gain)
            integral_time = self.tau
            print("PI Controller by IMC Tuning_PDCT: Kc = %f, tau_i = %f" % (controller_gain, integral_time))
        elif self.PID_type == "PID":
            controller_gain = (self.tau+(self.theta/2))/((tau_c+(self.theta/2))*process_gain)
            integral_time = self.tau+(self.theta/2)
            derivative_time = (self.tau*self.theta)/(2*self.tau+self.theta)     
            print("PID Controller by IMC Tuning_PDCT: Kc = %f, tau_i = %f, tau_d = %f" %
                 (controller_gain, integral_time, derivative_time))  
    
    def ITAE_1_tuning(self, process_gain=None):
        theta_tau_ratio = self.theta/self.tau
        if self.PID_type == "PI":
            controller_gain_set_point = 0.586*(theta_tau_ratio**(-0.916))/process_gain
            integral_time_set_point = self.tau/(1.030 - 0.165*theta_tau_ratio)
            controller_gain_disturbance = 0.859*(theta_tau_ratio**(-0.977))/process_gain
            integral_time_disturbance = self.tau/(0.674*(theta_tau_ratio**(-0.680)))
            print("PI controller by ITAE-1-setpoint tuning: Kc = %f, tau_i = %f" % (controller_gain_set_point, integral_time_set_point))
            print("PI Controller by ITAE-1-disturbance tuning: Kc = %f, tau_i = %f" % (controller_gain_disturbance, integral_time_disturbance))
        elif self.PID_type == "PID":
            controller_gain_set_point = 0.965*(theta_tau_ratio**(-0.850))/process_gain
            integral_time_set_point = self.tau/(0.796 - 0.1465*theta_tau_ratio)
            derivative_time_set_point = self.tau*0.308*(theta_tau_ratio**(0.929))
            controller_gain_disturbance = 1.357*(theta_tau_ratio**(-0.947))/process_gain
            integral_time_disturbance = self.tau/(0.842*(theta_tau_ratio**(-0.738)))
            derivative_time_disturbance = self.tau*0.381*(theta_tau_ratio**(0.995))
            print("PID Controller by ITAE-1-setpoint tuning: Kc = %f, tau_i = %f, tau_d = %f" %
                  (controller_gain_set_point, integral_time_set_point, derivative_time_set_point))
            print("PID Controller by ITAE-1-disturbance tuning: Kc = %f, tau_i = %f, tau_d = %f" %
                  (controller_gain_disturbance, integral_time_disturbance, derivative_time_disturbance))

In [3]:
#data load for training
learning_1st = pd.read_excel('DataFirst_rand_limit.xlsx')
pro_iden = FirstOrderProcessIdentification(loaded_data = learning_1st)
X_train, y_train, X_test, y_test, X_scale_minmax = pro_iden.data_preparation()
y_train_TC, y_train_TD = y_train[:,0:1], y_train[:,1:2]
y_test_TC, y_test_TD = y_test[:,0:1], y_test[:,1:2]

In [4]:
#Time Constant training
pro_iden_tau = FirstOrderProcessIdentification(learning_rate = 0.00001, momentum = 0.999,
                                               learning_object="tau", random_state=42)
pro_iden_tau.fit(X=X_train, y=y_train_TC, n_epochs=31)
y_pred_TC = pro_iden_tau.predict(X=X_test, y=y_test_TC)
print("predictive time constant:\n", y_pred_TC)
print("real time constant:\n", y_test_TC[:10])

0	batch data accuracy: -0.8982	batch data RMSE: 14.528848
1	batch data accuracy: 0.6237	batch data RMSE: 4.460631
2	batch data accuracy: 0.9134	batch data RMSE: 0.787401
3	batch data accuracy: 0.9915	batch data RMSE: 0.100834
4	batch data accuracy: 0.9840	batch data RMSE: 0.042804
5	batch data accuracy: 0.9988	batch data RMSE: 0.012111
6	batch data accuracy: 0.9992	batch data RMSE: 0.007317
7	batch data accuracy: 0.9997	batch data RMSE: 0.005131
8	batch data accuracy: 0.9997	batch data RMSE: 0.005750
9	batch data accuracy: 0.9998	batch data RMSE: 0.003334
10	batch data accuracy: 0.9997	batch data RMSE: 0.006868
11	batch data accuracy: 0.9997	batch data RMSE: 0.004211
12	batch data accuracy: 0.9999	batch data RMSE: 0.002275
13	batch data accuracy: 0.9999	batch data RMSE: 0.001283
14	batch data accuracy: 0.9998	batch data RMSE: 0.001930
15	batch data accuracy: 0.9998	batch data RMSE: 0.002219
16	batch data accuracy: 0.9998	batch data RMSE: 0.005988
17	batch data accuracy: 0.9998	batch da

In [5]:
#Time Delay training
pro_iden_theta = FirstOrderProcessIdentification(n_neurons=70, learning_rate = 0.00001, momentum = 0.999,
                                                 batch_size=10, learning_object="theta", random_state=42)
pro_iden_theta.fit(X=X_train, y=y_train_TD, n_epochs=32)
y_pred_TD = pro_iden_theta.predict(X=X_test, y=y_test_TD)
print("predictive time delay:\n", y_pred_TD)
print("real time delay:\n", y_test_TD[:10])

0	batch data accuracy: 0.9961	batch data RMSE: 0.016399
1	batch data accuracy: 0.9996	batch data RMSE: 0.003302
2	batch data accuracy: 0.9997	batch data RMSE: 0.001638
3	batch data accuracy: 0.9993	batch data RMSE: 0.006036
4	batch data accuracy: 0.9997	batch data RMSE: 0.003647
5	batch data accuracy: 0.9996	batch data RMSE: 0.004134
6	batch data accuracy: 0.9996	batch data RMSE: 0.004135
7	batch data accuracy: 0.9997	batch data RMSE: 0.003302
8	batch data accuracy: 0.9997	batch data RMSE: 0.003496
9	batch data accuracy: 0.9993	batch data RMSE: 0.001254
10	batch data accuracy: 0.9998	batch data RMSE: 0.001974
11	batch data accuracy: 0.9994	batch data RMSE: 0.002478
12	batch data accuracy: 0.9999	batch data RMSE: 0.000997
13	batch data accuracy: 0.9998	batch data RMSE: 0.002480
14	batch data accuracy: 0.9997	batch data RMSE: 0.001909
15	batch data accuracy: 0.9997	batch data RMSE: 0.002856
16	batch data accuracy: 0.9999	batch data RMSE: 0.000411
17	batch data accuracy: 0.9998	batch data

In [6]:
# data load for modeling
# The data used here is just data to work. Instead, change to the data you want to model.
operating_1st = pd.read_excel('DataFirst_Operating_rand_limit.xlsx')
pro_iden = FirstOrderProcessIdentification(loaded_data = operating_1st)
X_op, y_op, process_gain = pro_iden.operating_data_preparation(X_scale_minmax=X_scale_minmax)
y_op_TC, y_op_TD, = y_op[:,0:1], y_op[:,1:2]

In [7]:
tau_pred = pro_iden_tau.operating_predict(X=X_op, y=y_op_TC)
theta_pred = pro_iden_theta.operating_predict(X=X_op, y=y_op_TD)

print("\n"); print("predictive time constant:", tau_pred)
print("\n"); print("predictive time delay:", theta_pred)

INFO:tensorflow:Restoring parameters from ./1st_order_process_tau_learning.ckpt
INFO:tensorflow:Restoring parameters from ./1st_order_process_theta_learning.ckpt


predictive time constant: [[33.00825]]


predictive time delay: [[1.6332352]]


In [8]:
PID_tuning = PIDTuning(tau=tau_pred, theta=theta_pred, PID_type="PID")
process_gain = process_gain[:1]

PID_tuning.IMC_tuning_PIPC(process_gain=process_gain)
print("\n")
PID_tuning.IMC_tuning_PDCT(process_gain=process_gain, Condition="default value")
print("\n")
PID_tuning.ITAE_1_tuning(process_gain=process_gain)

PID Controller by IMC Tuning_PIPC: Kc = 0.575269, tau_i = 33.824867, tau_d = 0.796902


PID Controller by IMC Tuning_PDCT: Kc = 0.479391, tau_i = 33.824867, tau_d = 0.796902


PID Controller by ITAE-1-setpoint tuning: Kc = 0.431379, tau_i = 41.848747, tau_d = 0.622723
PID Controller by ITAE-1-disturbance tuning: Kc = 0.811992, tau_i = 4.263802, tau_d = 0.631686
